<a href="https://colab.research.google.com/github/Sanskar6877/BTP/blob/main/sbic_valid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text
import tensorflow_text as text
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
import json

# Choose the file from your local machine
uploaded = files.upload()

# Read the JSON file with explicit encoding specification
json_filename = list(uploaded.keys())[0]
with open(json_filename, 'r', encoding='utf-8', errors='replace') as file:
    data = json.load(file)

# Convert the loaded JSON data to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Create a binary 'hate' column based on the 'label_off' column
df['hate'] = df['label_off'].apply(lambda x: 0 if x == 'not offensive' else 1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df['post'], df['hate'], stratify=df['hate'])

# BERT preprocessor and encoder
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name='Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='Dense')(dropout)

# Build and compile the model
model = tf.keras.Model(inputs=[text_input], outputs=[outputs])

Metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=Metrics)

# Train the model
history = model.fit(X_train, y_train, epochs=10)

# Evaluate the model
model.evaluate(X_test, y_test)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.9 MB/s eta 0:00:00
Mounted at /content/drive


Saving sbic_valid.json to sbic_valid.json
Epoch 1/10
110/110 [==============================] - 51s 332ms/step - loss: 0.6970 - accuracy: 0.5430 - precision: 0.5401 - recall: 0.5136
Epoch 2/10
110/110 [==============================] - 38s 345ms/step - loss: 0.6615 - accuracy: 0.5999 - precision: 0.5987 - recall: 0.5800
Epoch 3/10
110/110 [==============================] - 38s 349ms/step - loss: 0.6354 - accuracy: 0.6476 - precision: 0.6475 - recall: 0.6314
Epoch 4/10
110/110 [==============================] - 37s 339ms/step - loss: 0.6214 - accuracy: 0.6519 - precision: 0.6492 - recall: 0.6447
Epoch 5/10
110/110 [==============================] - 38s 343ms/step - loss: 0.6129 - accuracy: 0.6610 - precision: 0.6611 - recall: 0.6459
Epoch 6/10
110/110 [==============================] - 38s 343ms/step - loss: 0.6050 - accuracy: 0.6730 - precision: 0.6688 - recall: 0.6719
Epoch 7/10
110/110 [==============================] - 38s 344ms/step - loss: 0.5949 - accuracy: 0.6831 - precision: 0.

[0.5654773116111755,
 0.7086546421051025,
 0.6814701557159424,
 0.7712305188179016]